<a href="https://colab.research.google.com/github/sinawrm/AppII-Entity-disambiguation/blob/main/notebooks/Final_Project_Demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 41.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 59.6 MB/s eta 0:00:00


In [ ]:
!pip install Whoosh 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.8/468.8 kB 8.3 MB/s eta 0:00:00


In [ ]:
import codecs
import pandas as pd
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
import spacy
import torch
from transformers import BertForQuestionAnswering
from transformers import BertTokenizer
from IPython.core.display import display, HTML # object displaying in different formats
from whoosh.index import * # whoosh: full-text indexing and searching
from whoosh.fields import *
from whoosh import qparser
import glob
import random
from tabulate import tabulate

In [ ]:
from google.colab import drive 
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd "/content/drive/MyDrive/LCT Courses/NLP Applications 2/APP II-FINAL PROJECT"

/content/drive/.shortcut-targets-by-id/1XEy1wqrAkSUBTAL6hPR4p7-Su4aMDubx/APP II-FINAL PROJECT


In [ ]:
nlp = spacy.load("en_core_web_sm")

In [ ]:
### Names Extraction function

def extract_name(paragraph):
    doc = nlp(paragraph)

    for entity in doc.ents:
      if entity.label_ == 'PERSON':
        return entity.text
    
    ##if no PERSON token found
    return ""

In [ ]:
###Indexing wikipedia summaries files

               
def index_summaries():

    
  # Schema definition:
  # - id: type ID, unique, stored; doc id in order given the passages file
  # - text: type TEXT processed by StemmingAnalyzer; not stored; content of the passage
  schema = Schema(id = ID(stored=True,unique=True),
                  text = TEXT(analyzer=analysis.StemmingAnalyzer()))


  if not os.path.exists("./demo/index"):
    os.mkdir("demo/index")
    
  ix = create_in("index", schema)
  writer = ix.writer() #run once! or restart runtime

  schema = Schema(id = ID(stored=True,unique=True),text = TEXT(analyzer=analysis.StemmingAnalyzer()))

  #create a list out of wikipedia summaries directory
  wikipedia_summaries = []
  wikipedia_summaries_files_list = []
  wikipedia_summaries_dict ="./data/wikipedia_summaries/"
  for filename in os.listdir(wikipedia_summaries_dict ):
      if filename.endswith('.txt'):  # Select only text files
          file_path = os.path.join(wikipedia_summaries_dict, filename)
          with open(file_path, 'r') as file:
              content = file.read()
              wikipedia_summaries.append(content)
          wikipedia_summaries_files_list.append(filename)

    ##add passages to index
  for i, summary in enumerate(wikipedia_summaries):
      writer.add_document(id=str(i),text= summary)

    # Save the added documents
  writer.commit()
  print("Index successfully created")

  # Sanity check
  print("Number of documents in the index: ", ix.doc_count())

  return ix, wikipedia_summaries, wikipedia_summaries_files_list
          

In [ ]:
##document retrival from wikipedia summaries
def retrieve_document(ix, paragraph, wikipedia_summaries, wikipedia_summaries_files_list, no_docs=1):
    score=0
    text=""

    # Open the searcher for reading the index. The default BM25 algorithm will be used for scoring
    with ix.searcher() as searcher:
        searcher = ix.searcher()

        # Define the query parser ('text' will be the default field to search), and set the input query
        q = qparser.QueryParser("text", ix.schema, group=qparser.OrGroup).parse(paragraph)
    
        # Search using the query q, and get the n_docs documents, sorted with the highest-scoring documents first
        results = searcher.search(q, limit=no_docs)
        # results is a list of dictionaries where each dictionary is the stored fields of the document

    predicted_summary = wikipedia_summaries[int(results[0]['id'])]
    ix_score = results[0].score
    predicted_filename = ""
    ##get predicted Wikipedia summary filename
    for i, text in enumerate(wikipedia_summaries):
      if text == predicted_summary:
        predicted_filename = wikipedia_summaries_files_list[i]
        break
    # return document and score of no_docs number of documents
    return predicted_filename, wikipedia_summaries[int(results[0]['id'])], results[0].score
  

In [ ]:
##Initialize QA model
class QA:
    def __init__(self):
        self.tokenizer = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')
        self.model = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

        print(f"QA object initialized!")
 ### question and answer function

In [ ]:
### question and answer function

def ask_questions(name, doc, QA_obj):

      
        # create queries
        queries=["What is the date of birth of "+name+"?", 
                 "What is the nationality of "+name+"?", 
                 "What is "+name+" doing?"]

        ##list that will contain answers for the queries, each item in the list will be a tuple(answer, start_score, end_score
        answers =[]

        ###loop over queries and get answer tokens

        for query in queries:
            
          #print(f"Tokenize query...")

          # Apply the tokenizer to the input text, treating them as a text-pair.
          input_ids = QA_obj.tokenizer.encode(query, doc,max_length=50)

          tokens = QA_obj.tokenizer.convert_ids_to_tokens(input_ids)
          
          # Search the input_ids for the first instance of the `[SEP]` token.
          sep_index = input_ids.index(QA_obj.tokenizer.sep_token_id)

          # The number of segment A tokens includes the [SEP] token istelf.
          num_seg_a = sep_index + 1

          # The remainder are segment B.
          num_seg_b = len(input_ids) - num_seg_a

          # Construct the list of 0s and 1s.
          segment_ids = [0]*num_seg_a + [1]*num_seg_b

          # There should be a segment_id for every input token.
          assert len(segment_ids) == len(input_ids)


          # Run our example through the model.
          outputs = QA_obj.model(torch.tensor([input_ids]), # The tokens representing our input text.
                                      token_type_ids=torch.tensor([segment_ids]), # The segment IDs to differentiate question from answer_text
                                      return_dict=True) 

          start_scores = outputs.start_logits
          end_scores = outputs.end_logits

          # Find the tokens with the highest `start` and `end` scores.
          answer_start = torch.argmax(start_scores)
          answer_end = torch.argmax(end_scores)

          # Combine the tokens in the answer and print it out.
          answer = ' '.join(tokens[answer_start:answer_end+1])

          # Start with the first token.
          answer = tokens[answer_start]

          # Select the remaining answer tokens and join them with whitespace.
          for i in range(answer_start + 1, answer_end + 1):
              
              # If it's a subword token, then recombine it with the previous token.
              if tokens[i][0:2] == '##':
                  answer += tokens[i][2:]
              
              # Otherwise, add a space then the token.
              else:
                  answer += ' ' + tokens[i]
 
          answers.append((answer,torch.max(start_scores).item(),torch.max(end_scores).item()))

        return answers

In [ ]:
def get_gold_answers(paragraph, paragraph_summary_all_df):
  matching_rows = paragraph_summary_all_df[paragraph_summary_all_df['paragraph'] == paragraph]
  if matching_rows.empty:
    return None
  
  matching_info = matching_rows[['name', 'wikipedia_file', 'wikipedia_summary', 'dob', 'nationality', 'occupation']]
  answers_dict = matching_info.to_dict(orient='records')[0]
    
  return answers_dict



In [ ]:
def print_wikimedia_table_gold(gold_answers):
  table_data = []
  for key, value in gold_answers.items():
      table_data.append([key, value])

  table_title = "Gold Information"

  table = tabulate(table_data, headers=["Field", "Value"], tablefmt="pipe")
  table_with_title = f"## {table_title}\n\n{table}"

  print(table_with_title)


In [ ]:
def print_wikimedia_table_predicted(predicted_name, predicted_filename, score, answers):
    table_data = [
        ["Predicted name", predicted_name],
        ["Predicted wikipedia_file (Score)", f"{predicted_filename} ({score})"]
    ]

    field_names = ["Predicted DoB", "Predicted Nationality", "Predicted Occupation"]

    for i, answer in enumerate(answers):
        field_name = field_names[i]
        field_data = str(answer)
        table_data.append([field_name, field_data])


    table_title = "Predicted Information"

    table = tabulate(table_data, headers=["Field", "Value"], tablefmt="pipe")
    table_with_title = f"## {table_title}\n\n{table}"

    print(table_with_title)


In [ ]:
###Initalizing all the important things (only once)

ix, wikipedia_summaries, wikipedia_summaries_files_list = index_summaries()

QA_obj =  QA()

paragraph_summary_all_df = pd.read_csv('./data/paragraph_summary_all.csv')


Index successfully created
Number of documents in the index:  200


QA object initialized!


In [ ]:
def demo():
  print("Welcome to Paz, Zina and Jehad Wikipedia QA Application!")
  print("This application implements NER - IR - QA Pipeline to extract information about persons from paragraphs")
  print("How to use: Select a paragraph and the program will detect the person mentioned and return their data of birth, nationality and occupation")
  print("Choose one of the paragraphs below:")

  for index, row in paragraph_summary_all_df.iterrows():
    paragraph = row['paragraph']
    print(f"{index}. {paragraph}")

  print("enter the index of the paragraph that you want to extract information about it")
  selected_paragraph_no = input("enter paraagraph index: ")
  
  print("You selected paragraph", selected_paragraph_no)
  paragraph = paragraph_summary_all_df.loc[int(selected_paragraph_no), 'paragraph']
  print("extracting person name from paragraph...")
  predicted_name = extract_name(paragraph)
  print("matching wikipedia summary....")
  predicted_filename, predicted_wikipedia_summary, score = retrieve_document(ix, paragraph, wikipedia_summaries, wikipedia_summaries_files_list, no_docs=1)
  print("asking question about date of birth, nationality and occupation....")
  #predicted_answers = ask_questions(predicted_name, predicted_wikipedia_summary, QA_obj)
  #print(predicted_name,QA_obj)
  predicted_answers = ask_questions(predicted_name, predicted_wikipedia_summary, QA_obj)
  print("getting gold answeres (for evaluation)....")
  gold_answers =  get_gold_answers(paragraph, paragraph_summary_all_df)

  print("Here are the results:")
  print_wikimedia_table_predicted(predicted_name, predicted_filename, score, predicted_answers)
  print("\n")
  print_wikimedia_table_gold(gold_answers)
  print("\n")
  








In [ ]:
is_exit = False
while not is_exit:
  demo()
  while True:
    user_input = input("Enter \"exit\" or \"restart\": ")
    if user_input == "exit":
      is_exit = True
      break
    elif user_input == "restart":
      break
    else:
      print("sorry your input is invalid.")



Welcome to Paz, Zina and Jehad Wikipedia QA Application!
This application implements NER - IR - QA Pipeline to extract information about persons from paragraphs
How to use: Select a paragraph and the program will detect the person mentioned and return their data of birth, nationality and occupation
Choose one of the paragraphs below:
0. Barack Obama, the charismatic leader made history with his groundbreaking presidency. Known for his eloquence and inspiring speeches, Obama brought a message of hope and change to the American people. During his tenure, he implemented several key initiatives, such as the Affordable Care Act, aiming to provide healthcare coverage to millions of Americans. Obama's leadership was also marked by his efforts to improve international relations and promote diplomacy. His remarkable journey from a community organizer to the leader of the free world continues to inspire generations and serves as a testament to the power of perseverance and determination.
1. In t

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


getting gold answeres (for evaluation)....
Here are the results:
## Predicted Information

| Field                            | Value                                                          |
|:---------------------------------|:---------------------------------------------------------------|
| Predicted name                   | Priyanka Chopra                                                |
| Predicted wikipedia_file (Score) | Priyanka Chopra.txt (70.76759844114297)                        |
| Predicted DoB                    | ('18 july 1982', 8.963935852050781, 10.308828353881836)        |
| Predicted Nationality            | ('indian', 6.355964660644531, 6.901256561279297)               |
| Predicted Occupation             | ('actress and producer', 5.640876770019531, 6.806397438049316) |


## Gold Information

| Field             | Value                                                                                                                                                 

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


You selected paragraph 3
extracting person name from paragraph...
matching wikipedia summary....
asking question about date of birth, nationality and occupation....


Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


getting gold answeres (for evaluation)....
Here are the results:
## Predicted Information

| Field                            | Value                                              |
|:---------------------------------|:---------------------------------------------------|
| Predicted name                   | Malala Yousafzai                                   |
| Predicted wikipedia_file (Score) | Malala Yousafzai.txt (94.6178717393359)            |
| Predicted DoB                    | ('urdu', -0.6127598881721497, -0.7066386342048645) |
| Predicted Nationality            | ('pashto', 4.804942607879639, 4.985958099365234)   |
| Predicted Occupation             | ('urdu', -3.81141996383667, -2.5089845657348633)   |


## Gold Information

| Field             | Value                                                                                                                                                                                                                                     